In [1]:
from pathlib import Path
import csv
from itertools import groupby
import h5py
import numpy as np
import sklearn
from sklearn.cluster import KMeans
from sklearn.svm import SVC
# from sklearn.linear_model import LogisticRegression
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import random
import matplotlib
from matplotlib import pyplot as plt

# Configuration & Utilities

In [2]:
random.seed(42)
COLORS = ["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", "#9467bd", "#8c564b", "#e377c2", "#7f7f7f", "#bcbd22", "#17becf"]
ANNOTATIONS_PATH = "/media/xtrem/data/experiments/nicolingua-0001-language-id/language-id-annotations/metadata.csv"
FEATURE_DIRS = [
    '/media/xtrem/data/experiments/nicolingua-0001-language-id/wav2vec_features-c',
    '/media/xtrem/data/experiments/nicolingua-0001-language-id/wav2vec_features-z',
    '/media/xtrem/data/experiments/nicolingua-0001-language-id/retrained-wav2vec_features-c',
    '/media/xtrem/data/experiments/nicolingua-0001-language-id/retrained-wav2vec_features-z'
]

In [3]:
annotation_specification = {
    0: {
        'id': 0,
        'label': "maninka",
        'required_tags': set(['ct-speech', 'lng-maninka']),
        'forbidden_tags':  set(['lng-susu', 'lng-pular'])
    },
    1: {
        'id': 1,
        'label': "susu",
        'required_tags': set(['ct-speech', 'lng-susu']),
        'forbidden_tags':  set(['lng-maninka', 'lng-pular'])
    },
    2: {
        'id': 2,
        'label': "pular",
        'required_tags': set(['ct-speech', 'lng-pular']),
        'forbidden_tags':  set(['lng-susu', 'lng-maninka'])
    }
}

In [4]:
def to_user_friendly_feature_name(fv_name):
    name = fv_name \
        .replace("features-", "") \
        .replace("wav2vec_", "") \
        .replace("average", "avg") \
        .replace("timestep", "T") \
        .replace("c.", "Context") \
        .replace("z.", "Latent")
    return name

# Load annotations

In [5]:
def load_annotations(a_file_path, a_specification):
    with open(ANNOTATIONS_PATH) as f:
        reader = csv.DictReader(f)
        for row in reader:
            tag_set = set([t.strip() for t in row['tags'].split(";")])
            for label in annotation_specification.keys():
                spec = annotation_specification[label]
                if spec['required_tags'].issubset(tag_set):
                    if spec['forbidden_tags'].isdisjoint(tag_set):
                        yield row['file'], label
                        break

data = list(load_annotations(ANNOTATIONS_PATH, annotation_specification))
audio_files, audio_labels = zip(*data)

## Inspect label counts

In [6]:
def inspect_label_counts():
    for label in annotation_specification:
        count = len([l for l in audio_labels if l == label])
        print("{:10} ({}): {}".format(
            annotation_specification[label]['label'],
            label, 
            count
        ))
inspect_label_counts()

maninka    (0): 114
susu       (1): 32
pular      (2): 28


## Balance data

In [7]:
count_per_class = 28
data = list(load_annotations(ANNOTATIONS_PATH, annotation_specification))
balanced_data = []
for label in annotation_specification:
    balanced_data.extend([d for d in data if d[1] == label][:count_per_class])
audio_files, audio_labels = zip(*balanced_data)

In [8]:
inspect_label_counts()

maninka    (0): 28
susu       (1): 28
pular      (2): 28


# Prepare 10 cross validation folds

In [9]:
TRAIN_PERCENT = .6
FOLD_COUNT = 10

n = len(audio_files)
n_train = int(np.ceil(n * .6))
n_test = n - n_train
all_indices = range(n)

cv_folds = {}
train_count_by_index = {i:0 for i in all_indices}
test_count_by_index = {i:0 for i in all_indices}

for fold_index in range(FOLD_COUNT):
    fold_rsampler = np.random.RandomState(seed=fold_index)
    train_index_set = set(fold_rsampler.choice(all_indices, n_train, replace=False))
    test_index_set = set(all_indices).difference(train_index_set)
        
    cv_folds[fold_index] = {
        'train_indices': sorted(list(train_index_set)),
        'test_indices': sorted(list(train_index_set)),
    }


# Load features

In [10]:
def load_features(audio_files, features_input_dir):
    id_list = []
    features_list = []

    for audio_file_name in audio_files:
        feature_file_name = audio_file_name.replace(".wav", ".h5context")
        feature_path = Path(features_input_dir) / feature_file_name
        with h5py.File(feature_path, 'r') as f:
            features_shape = f['info'][1:].astype(int)
            features = np.array(f['features'][:]).reshape(features_shape)
            # features = pool_feature_last_seq(features)
            features_list.append(features)
    return features_list

In [11]:
raw_features = {}
for feature_dir in FEATURE_DIRS:
    feature_name = Path(feature_dir).stem
    raw_features[feature_name] = load_features(audio_files, feature_dir)

## Inspect feature shapes

In [12]:
for feature_name in raw_features.keys():
    print("feature_name: {}. feature shape: {}".format(
        to_user_friendly_feature_name(feature_name),
        raw_features['wav2vec_features-c'][0].shape
    ))

feature_name: c. feature shape: (2998, 512)
feature_name: z. feature shape: (2998, 512)
feature_name: retrained-c. feature shape: (2998, 512)
feature_name: retrained-z. feature shape: (2998, 512)


## Extract feature vectors

In [13]:
def extract_last_timestep_features(raw_features):
    return raw_features[-1, :]

def extract_neuron_average_features(raw_features):
    return np.mean(raw_features, axis=0)

def identity(x):
    return x

feature_extractors = {
    'last_timestep': extract_last_timestep_features,
    'neuron_average': extract_neuron_average_features,
    'raw_features': identity
}

In [14]:
feature_vectors = {}
for feature_name in raw_features.keys():
    for feature_extractor_name in feature_extractors.keys():
        fv_name = f"{feature_name}__{feature_extractor_name}"
        feature_vectors[fv_name] = []
        for f in raw_features[feature_name]:
            feature_vectors[fv_name].append(
                feature_extractors[feature_extractor_name](f)
            )
            
        feature_vectors[fv_name] = np.array(feature_vectors[fv_name])

## Inspect feature vectors

In [15]:
for fvname in feature_vectors.keys():
    print(fvname, feature_vectors[fvname][0].shape)

wav2vec_features-c__last_timestep (512,)
wav2vec_features-c__neuron_average (512,)
wav2vec_features-c__raw_features (2998, 512)
wav2vec_features-z__last_timestep (512,)
wav2vec_features-z__neuron_average (512,)
wav2vec_features-z__raw_features (2998, 512)
retrained-wav2vec_features-c__last_timestep (512,)
retrained-wav2vec_features-c__neuron_average (512,)
retrained-wav2vec_features-c__raw_features (2998, 512)
retrained-wav2vec_features-z__last_timestep (512,)
retrained-wav2vec_features-z__neuron_average (512,)
retrained-wav2vec_features-z__raw_features (2998, 512)


# Classification Models

In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from pytorch_model_summary import summary
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim

In [17]:
class LangIdConvNet(nn.Module):
    def __init__(self):
        super(LangIdConvNet, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=512, out_channels=8, kernel_size=3)
        self.drop1 = nn.Dropout(p=0.2)
        self.pool1 = nn.MaxPool1d(kernel_size=2, stride=2)
        
        self.conv2 = nn.Conv1d(in_channels=8, out_channels=8, kernel_size=3)
        self.drop2 = nn.Dropout(p=0.2)
        self.pool2 = nn.MaxPool1d(kernel_size=2, stride=2)
        
        self.conv3 = nn.Conv1d(in_channels=8, out_channels=8, kernel_size=3)
        self.drop3 = nn.Dropout(p=0.2)
        self.pool3 = nn.MaxPool1d(kernel_size=2, stride=2)
        
        self.conv4 = nn.Conv1d(in_channels=8, out_channels=3, kernel_size=3)
        self.pool4 = nn.MaxPool1d(kernel_size=2, stride=2)
        
    def forward(self, x):
        x = x.permute(0, 2, 1)
        x = self.conv1(x)
        x = F.elu(x)
        x = self.drop1(x)
        x = self.pool1(x)
        
        x = self.conv2(x)
        x = F.elu(x)
        x = self.drop2(x)
        x = self.pool2(x)
        
        x = self.conv3(x)
        x = F.elu(x)
        x = self.drop3(x)
        x = self.pool3(x)
        
        x = self.conv4(x)
        x = F.elu(x)
        x = self.pool4(x)
        
        x = torch.mean(x, dim=2)
        
        return x
        

# Train Classification Models

In [18]:
def get_data_for_fold(fold_id, batch_size):
    train_indices = cv_folds[fold_id]['train_indices']
    test_indices = cv_folds[fold_id]['test_indices']

    train_x = np.take(raw_features['wav2vec_features-c'], train_indices, axis=0)
    train_y = np.take(audio_labels, train_indices, axis=0)

    test_x = np.take(raw_features['wav2vec_features-c'], test_indices, axis=0)
    test_y = np.take(audio_labels, test_indices, axis=0)
    
    return train_x, train_y, test_x, test_y

    
def get_loaders_for_fold(fold_id, batch_size):
    
    train_x, train_y, test_x, test_y = get_data_for_fold(fold_id, batch_size)
    
    train_dataset = TensorDataset(
        torch.tensor(train_x), 
        torch.tensor(train_y)
    )

    train_loader = DataLoader(train_dataset, batch_size=batch_size)

    test_dataset = TensorDataset(
        torch.tensor(test_x), 
        torch.tensor(test_y)
    )

    test_loader = DataLoader(test_dataset, batch_size=batch_size)
    
    return train_loader, test_loader


def get_predictions_for_logits(logits):
    probs = F.softmax(logits, dim=1)
    return torch.argmax(probs, dim=1)

In [ ]:
model = LangIdConvNet()

train_loader, test_loader = get_loaders_for_fold(0, 10)

print(summary(LangIdConvNet(), torch.zeros((10, 2998, 512)), show_input=False))


    

optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

for epoch in range(500):
    model.train()
    train_loss = 0
    pred_train_classes = []
    true_train_classes = []
    
    for batch_idx, (x, y) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(x)
        pred_train_classes.extend(
            get_predictions_for_logits(outputs)
        )
        true_train_classes.extend(y)
        loss = criterion(outputs, y)
        
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        
    train_loss = train_loss/len(train_loader)
    train_acc = sklearn.metrics.accuracy_score(true_train_classes, pred_train_classes)
    
    
    pred_test_classes = []
    true_test_classes = []
    model.eval()
    test_loss = 0
    for batch_idx, (x, y) in enumerate(test_loader):
        outputs = model(x)
        
        pred_test_classes.extend(
            get_predictions_for_logits(outputs)
        )
        
        true_test_classes.extend(y)
        
        loss = criterion(outputs, y)
        test_loss += loss.item()
        
        
    test_loss = test_loss / len(test_loader)
    test_acc = sklearn.metrics.accuracy_score(true_test_classes, pred_test_classes)
    
    
    print(f"Epoch: {epoch}. Train Loss: {train_loss:0.4}. Test Loss: {test_loss:0.4}. Train Acc: {train_acc:0.4}. Test Acc:{test_acc:0.4}")
    

-----------------------------------------------------------------------
      Layer (type)        Output Shape         Param #     Tr. Param #
          Conv1d-1       [10, 8, 2996]          12,296          12,296
         Dropout-2       [10, 8, 2996]               0               0
       MaxPool1d-3       [10, 8, 1498]               0               0
          Conv1d-4       [10, 8, 1496]             200             200
         Dropout-5       [10, 8, 1496]               0               0
       MaxPool1d-6        [10, 8, 748]               0               0
          Conv1d-7        [10, 8, 746]             200             200
         Dropout-8        [10, 8, 746]               0               0
       MaxPool1d-9        [10, 8, 373]               0               0
         Conv1d-10        [10, 3, 371]              75              75
      MaxPool1d-11        [10, 3, 185]               0               0
Total params: 12,771
Trainable params: 12,771
Non-trainable params: 0
------

Epoch: 84. Train Loss: 0.6001. Test Loss: 0.6128. Train Acc: 0.7059. Test Acc:0.7451
Epoch: 85. Train Loss: 0.5906. Test Loss: 0.6087. Train Acc: 0.7647. Test Acc:0.7451
Epoch: 86. Train Loss: 0.5868. Test Loss: 0.6039. Train Acc: 0.7647. Test Acc:0.7647
Epoch: 87. Train Loss: 0.5849. Test Loss: 0.6011. Train Acc: 0.7647. Test Acc:0.7451
Epoch: 88. Train Loss: 0.5846. Test Loss: 0.5978. Train Acc: 0.7647. Test Acc:0.7647
Epoch: 89. Train Loss: 0.5678. Test Loss: 0.592. Train Acc: 0.7451. Test Acc:0.7647
Epoch: 90. Train Loss: 0.5721. Test Loss: 0.5897. Train Acc: 0.7843. Test Acc:0.7647
Epoch: 91. Train Loss: 0.5653. Test Loss: 0.5839. Train Acc: 0.8039. Test Acc:0.7451
Epoch: 92. Train Loss: 0.5619. Test Loss: 0.5807. Train Acc: 0.7647. Test Acc:0.7451
Epoch: 93. Train Loss: 0.5558. Test Loss: 0.5767. Train Acc: 0.8039. Test Acc:0.7451
Epoch: 94. Train Loss: 0.5504. Test Loss: 0.5714. Train Acc: 0.7451. Test Acc:0.7451
Epoch: 95. Train Loss: 0.5541. Test Loss: 0.5684. Train Acc: 0.764

Epoch: 181. Train Loss: 0.07773. Test Loss: 0.1373. Train Acc: 1.0. Test Acc:1.0
Epoch: 182. Train Loss: 0.07404. Test Loss: 0.1473. Train Acc: 1.0. Test Acc:1.0
Epoch: 183. Train Loss: 0.07033. Test Loss: 0.1324. Train Acc: 1.0. Test Acc:1.0
Epoch: 184. Train Loss: 0.06789. Test Loss: 0.1377. Train Acc: 0.9804. Test Acc:1.0
Epoch: 185. Train Loss: 0.06544. Test Loss: 0.1286. Train Acc: 1.0. Test Acc:1.0
Epoch: 186. Train Loss: 0.06628. Test Loss: 0.132. Train Acc: 1.0. Test Acc:1.0
Epoch: 187. Train Loss: 0.05913. Test Loss: 0.1234. Train Acc: 1.0. Test Acc:1.0
Epoch: 188. Train Loss: 0.06038. Test Loss: 0.1281. Train Acc: 1.0. Test Acc:1.0
Epoch: 189. Train Loss: 0.055. Test Loss: 0.1171. Train Acc: 1.0. Test Acc:1.0
Epoch: 190. Train Loss: 0.05943. Test Loss: 0.1225. Train Acc: 1.0. Test Acc:1.0
Epoch: 191. Train Loss: 0.0521. Test Loss: 0.1164. Train Acc: 1.0. Test Acc:1.0
Epoch: 192. Train Loss: 0.05181. Test Loss: 0.1152. Train Acc: 1.0. Test Acc:1.0
Epoch: 193. Train Loss: 0.051

In [ ]:
for fold_index in cv_folds:
    x_train = 
    print(fold_index)

In [ ]:
def get_sample_weights(y):
    count_per_class = {k: len(list(g)) for k, g in groupby(sorted(y))}
    class_count = len(count_per_class)
    weight_per_class = {k: 1/class_count/c for k, c in count_per_class.items()}
    return [weight_per_class[yi] for yi in y]

In [ ]:
train_size = len(audio_labels) // 2

plt.figure(figsize=(30, 15))

for index, fv_name in enumerate(feature_vectors.keys()):
    X = feature_vectors[fv_name]
    
    x_train = X[:train_size]
    y_train = audio_labels[:train_size]

    x_test = X[train_size:]
    y_test = audio_labels[train_size:]

    svc = SVC(kernel="poly")
    svc.fit(x_train, y_train)

    train_acc = svc.score(x_train, y_train, sample_weight=get_sample_weights(y_train))
    test_acc = svc.score(x_test, y_test, sample_weight=get_sample_weights(y_test))
    print(fv_name, train_acc, test_acc)
    
    print(f'plt.subplot({2}, {int(np.ceil(len(feature_vectors.keys())/2))}, {index+1})')
    plt.subplot(2, int(np.ceil(len(feature_vectors.keys())/2)), index+1)
    
    projected_x = TSNE(n_components=2).fit_transform(feature_vectors['retrained-wav2vec_features-c__neuron_average'])
    plt.scatter(projected_x[:, 0], projected_x[:, 1], c=[COLORS[l] for l in audio_labels])
    title = to_user_friendly_feature_name(fv_name)
    
    title = f"{title} SVM({train_acc:.02%}, {test_acc:.02%})"
    
    plt.title(title)


print(train_acc, test_acc)

In [ ]:
import torch